In [1]:
import sys
sys.path.insert(0,'/home/djl34/kl_git/scripts')
from notebook_imports import *

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
client = Client()

In [3]:
chrom = "*"
filename = os.path.join(KL_data_dir, f"results/footprints/zscore/binary/{chrom}.tsv")
ddf = dd.read_csv(filename, sep = "\t", dtype={'Spliceai_info': 'object'})
df_binary = ddf.compute()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [4]:
def get_zscore(obs, exp):
    if (obs - exp) < 0:
        return (((obs - exp)**2)/exp)**0.5
    else:
        return -1 * (((obs - exp)**2)/exp)**0.5

In [5]:
df_binary["zscore_binary"] = df_binary.apply(lambda row: get_zscore(row["polymorphic"], row["polymorphic_expected"]), axis=1)

In [6]:
filename = os.path.join(KL_data_dir, f"results/footprints/zscore/AC/{chrom}.tsv")
ddf = dd.read_csv(filename, sep = "\t", dtype={'Spliceai_info': 'object'})
df_ac = ddf.compute()

In [7]:
# df_ac["zscore_ac_chi"] = df_ac.apply(lambda row: get_zscore(row["AC_total"], row["expected_AC"]), axis=1)
df_ac["zscore_ac"] = (df_ac["AC_total"] - df_ac["expected_AC"])/np.sqrt(df_ac["AC_var"])

In [8]:
df_ac.head()

,footprint_identifier,mu,expected_AC,AC_var,AC_total,AN_mean,sites,zscore_ac
0,1.102814.3,4.278,564.329566,5.723804e+06,21.0,550917.515152,33,-0.227102
1,1.102814.44,11.287,1496.473430,1.509851e+07,432.0,549097.975309,81,-0.273948
2,1.102814.47,16.900,2219.699413,2.249524e+07,134.0,550766.588235,136,-0.439751
3,1.102821.4,6.963,902.101016,9.095739e+06,473.0,552212.023810,84,-0.142279
4,1.1029632.1,6.412,833.497037,8.489607e+06,197.0,552300.055556,90,-0.218450


In [9]:
filename = os.path.join(KL_data_dir, f"results/footprints/phyloP/filtered_common_{chrom}.tsv")
ddf = dd.read_csv(filename, sep = "\t")
df_phyloP = ddf.compute()

filename = os.path.join(KL_data_dir, f"results/footprints/denovo/filtered_common_{chrom}.tsv")
ddf = dd.read_csv(filename, sep = "\t")
df_denovo = ddf.compute()

In [10]:
df_binary = df_binary.drop(["sites"], axis = 1)

In [11]:
df_merged = df_binary.merge(df_ac, on = "footprint_identifier", how = "outer")

In [12]:
df = df_merged.merge(df_phyloP, on = "footprint_identifier", how = "left")
df = df.merge(df_denovo, on = "footprint_identifier", how = "left")

In [17]:
df["denovo_control"] = df["denovo_an_control"] + df["denovo_halldorsson_control"]

In [13]:
df

,footprint_identifier,polymorphic,polymorphic_expected,zscore_binary,mu,expected_AC,AC_var,AC_total,AN_mean,sites,zscore_ac,phyloP,phyloP_pos,phyloP_max,phyloP_median,denovo_an_case,denovo_an_control,denovo_halldorsson_control
0,1.102814.3,5,6.968790,0.745797,4.278,564.329566,5.723804e+06,21.0,550917.515152,33,-0.227102,-0.672000,0.102909,0.568,-0.3530,0.0,0.0,0.0
1,1.102814.44,23,17.488485,-1.317938,11.287,1496.473430,1.509851e+07,432.0,549097.975309,81,-0.273948,-0.962383,0.155370,0.545,-0.4060,0.0,0.0,0.0
2,1.102814.47,34,27.470030,-1.245896,16.900,2219.699413,2.249524e+07,134.0,550766.588235,136,-0.439751,-0.580324,0.056081,0.545,0.0520,0.0,0.0,0.0
3,1.102821.4,10,11.816054,0.528315,6.963,902.101016,9.095739e+06,473.0,552212.023810,84,-0.142279,-0.301964,0.278643,1.083,-0.0540,0.0,0.0,0.0
4,1.1029632.1,10,10.819069,0.249015,6.412,833.497037,8.489607e+06,197.0,552300.055556,90,-0.218450,0.230933,0.340833,0.572,0.4375,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4088937,9.278626.1,1,0.246404,-1.518152,0.133,16.774453,1.698792e+05,17.0,551451.333333,3,0.000547,0.010000,0.010000,0.010,0.0100,0.0,0.0,0.0
4088938,9.337069.1,0,0.232256,0.481929,0.126,15.802947,1.601476e+05,0.0,551394.000000,3,-0.039489,0.271000,0.271000,0.271,0.2710,0.0,0.0,0.0
4088939,9.603679.1,1,0.564077,-0.580418,0.332,43.105009,4.387943e+05,1.0,551818.666667,3,-0.063563,0.261000,0.261000,0.261,0.2610,0.0,0.0,0.0
4088940,9.7054906.6,0,0.475155,0.689315,0.277,35.760237,3.644968e+05,0.0,552290.000000,3,-0.059232,1.081000,1.081000,1.081,1.0810,0.0,0.0,0.0


In [14]:
total_length = len(df)

column_name = "zscore_binary"

print(df[column_name].min())
print(df[column_name].max())
print(df[column_name].mean())
print(len(df[df[column_name] < -1]))
print(len(df[df[column_name] < -0.5]))

-19.866397264334683
9.840374496391481
0.04965003833308691
533578
1059060


In [15]:
column_name = "zscore_ac"

print(df_merged[column_name].min())
print(df_merged[column_name].max())
print(df_merged[column_name].mean())
print(len(df_merged[df_merged[column_name] < -1]))
print(len(df_merged[df_merged[column_name] < -0.5]))

-2.7309230501550448
57.304889797978035
-0.0343337890655595
124
5567


In [13]:
df_footprint_mean = pd.DataFrame(df[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean(), columns= ["all"])

In [14]:
df[["denovo_an_case", "denovo_an_control", "denovo_halldorsson_control"]].sum()

denovo_an_case                3729.0
denovo_an_control             3648.0
denovo_halldorsson_control    5303.0
dtype: float64

## do it for top set of footprints

In [15]:
df_zscore = df.sort_values("zscore_ac", ascending = True)
df_zscore_ac_1000 = df_zscore[:1000]
df_zscore_ac_1000_rest = df_zscore[1000:]

df_zscore = df.sort_values("zscore_binary", ascending = True)
df_zscore_binary_1000 = df_zscore[:1000]
df_zscore_binary_1000_rest = df_zscore[1000:]



In [16]:
print(df_zscore_ac_1000[["denovo_an_case", "denovo_an_control"]].sum())
print(df_zscore_ac_1000_rest[["denovo_an_case", "denovo_an_control"]].sum())

denovo_an_case       9.0
denovo_an_control    3.0
dtype: float64
denovo_an_case       3720.0
denovo_an_control    3645.0
dtype: float64


In [17]:
print(df_zscore_binary_1000[["denovo_an_case", "denovo_an_control"]].sum())
print(df_zscore_binary_1000_rest[["denovo_an_case", "denovo_an_control"]].sum())

denovo_an_case       14.0
denovo_an_control    18.0
dtype: float64
denovo_an_case       3715.0
denovo_an_control    3630.0
dtype: float64


In [21]:
df_footprint_mean = pd.DataFrame(df[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean(), columns= ["all"])
df_footprint_mean["AC top 1000"] = df_zscore_ac_1000[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()
df_footprint_mean["Binary top 1000"] = df_zscore_binary_1000[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

df_footprint_mean

,all,AC top 1000,Binary top 1000
phyloP,0.500888,1.421131,0.198757
phyloP_pos,0.839108,1.754884,2.077967
phyloP_max,2.503774,4.841373,4.170007
phyloP_median,0.547084,1.495829,0.873036


In [59]:
df_zscore_ac_1000.to_csv("top_1000_footprints.tsv", sep = "\t", index = None)

### top 5000

In [18]:
df_zscore = df.sort_values("zscore_ac", ascending = True)
df_zscore_ac_5000 = df_zscore[:5000]
df_zscore_ac_5000_rest = df_zscore[5000:]

df_zscore = df.sort_values("zscore_binary", ascending = True)
df_zscore_binary_5000 = df_zscore[:5000]
df_zscore_binary_5000_rest = df_zscore[5000:]



In [22]:
df_zscore_ac_5000.to_csv("top_5000_footprints.tsv", sep = "\t", index = None)

In [19]:
print(df_zscore_ac_5000[["denovo_an_case", "denovo_an_control"]].sum())
print(df_zscore_ac_5000_rest[["denovo_an_case", "denovo_an_control"]].sum())


denovo_an_case       33.0
denovo_an_control    20.0
dtype: float64
denovo_an_case       3696.0
denovo_an_control    3628.0
dtype: float64


In [20]:
print(df_zscore_binary_5000[["denovo_an_case", "denovo_an_control"]].sum())
print(df_zscore_binary_5000_rest[["denovo_an_case", "denovo_an_control"]].sum())

denovo_an_case       26.0
denovo_an_control    32.0
dtype: float64
denovo_an_case       3703.0
denovo_an_control    3616.0
dtype: float64


In [22]:
df_footprint_mean = pd.DataFrame(df[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean(), columns= ["all"])
df_footprint_mean["AC top 5000"] = df_zscore_ac_5000[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()
df_footprint_mean["Binary top 5000"] = df_zscore_binary_5000[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

df_footprint_mean

,all,AC top 5000,Binary top 5000
phyloP,0.500888,1.011473,0.394984
phyloP_pos,0.839108,1.397255,1.314178
phyloP_max,2.503774,4.398929,3.298868
phyloP_median,0.547084,1.093315,0.661085


In [48]:
print(df_zscore_binary_5000["denovo_an_case"].sum())
print(df_zscore_binary_5000["denovo_an_control"].sum())

26.0
32.0


In [51]:
df_zscore_binary_5000[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

phyloP           0.394984
phyloP_pos       1.314178
phyloP_max       3.298868
phyloP_median    0.661085
dtype: float64

### Z_score < -1

In [40]:
df_zscore_1 = df[df["zscore_ac"] < -1]

print(df_zscore_1["denovo_an_case"].sum())
print(df_zscore_1["denovo_an_control"].sum())

df_footprint_mean["Zscore < -1"] = df_zscore_1[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

0.0
0.0


In [57]:
df_footprint_mean

,all,Zscore < 1
phyloP,0.500888,2.104707
phyloP_pos,0.839108,2.377669
phyloP_max,2.503774,5.509202
phyloP_median,0.547084,2.128012


### Z_score < 0.5

In [43]:
df_zscore_1 = df[df["zscore_ac"] < -0.5]

print(df_zscore_1["denovo_an_case"].sum())
print(df_zscore_1["denovo_an_control"].sum())

df_footprint_mean = pd.DataFrame(df[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean(), columns= ["all"])
df_footprint_mean["Zscore < -0.5"] = df_zscore_1[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

35.0
22.0


In [44]:
df_footprint_mean

,all,Zscore < -0.5
phyloP,0.500888,0.988912
phyloP_pos,0.839108,1.377093
phyloP_max,2.503774,4.366154
phyloP_median,0.547084,1.073150


In [74]:
df_zscore["denovo_an_control"].sum()

22.0

## other cutoffs

In [87]:
df_zscore = df[df["Z_score"] < - 0.7]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_case"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())

df_footprint_mean["Zscore < -0.7"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

435
0.7295795988401905
5.0
0.0


In [82]:
df_zscore = df[df["Z_score"] < - 0.8]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())

df_footprint_mean["Zscore < -0.8"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

250
0.0
0.0


In [88]:
df_zscore = df[df["Z_score"] < - 0.6]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_case"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_control"].sum())


df_footprint_mean["Zscore < -0.6"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

1156
0.8410047203935402
10.0
0.3364018881574161
4.0


In [86]:
df_footprint_mean

,all,Zscore < -1,Zscore < -0.8,Zscore < -0.7,Zscore < -0.6,Zscore < -0.5
phyloP,0.500888,2.104707,1.978092,1.805940,1.346052,0.988912
phyloP_pos,0.839108,2.377669,2.247573,2.098740,1.687538,1.377093
phyloP_max,2.503774,5.509202,5.239268,5.156821,4.770325,4.366154
phyloP_median,0.547084,2.128012,2.016986,1.861807,1.417084,1.073150
